# 자체 데이터를 사용한 Azure 채팅 완료 모델(미리 보기)
이 예에서는 자체 데이터로 Azure OpenAI 서비스 모델을 사용하는 방법을 보여 줍니다. 이 기능은 현재 미리 보기 중입니다.

데이터에서 Azure OpenAI를 사용하면 모델을 학습하거나 미세 조정할 필요 없이 데이터에서 GPT-3.5-Turbo 및 GPT-4와 같은 지원되는 채팅 모델을 실행할 수 있습니다. 데이터에서 모델을 실행하면 데이터를 기반으로 채팅하고 더 정확하고 빠르게 데이터를 분석할 수 있습니다. 데이터에서 Azure OpenAI의 주요 이점 중 하나는 대화형 AI의 콘텐츠를 맞춤 설정할 수 있다는 점입니다. 모델은 응답을 지원하기 위해 특정 원본에 액세스하고 이를 참조할 수 있으므로 사전 학습된 지식뿐만 아니라 지정된 데이터 원본에서 사용 가능한 최신 정보에 기반하여 답변합니다. 또한 이러한 근거 데이터는 모델이 오래되거나 부정확한 정보를 기반으로 답변을 생성하는 것을 방지하는 데 도움이 됩니다.

Azure Cognitive Search를 사용하여 자체 데이터에 Azure OpenAI를 사용하면 지식 검색을 위한 사용자 지정 가능한 사전 빌드된 솔루션을 제공하며, 이를 통해 대화형 AI 애플리케이션을 빌드할 수 있습니다. 지식 검색 및 시맨틱 검색을 위한 다른 방법을 보려면 [벡터 데이터베이스](https://github.com/openai/openai-cookbook/tree/main/examples/vector_databases)에 대한 쿡북 예제를 확인하세요.

## 작동 방식

[사용자 데이터의 Azure OpenAI](https://learn.microsoft.com/azure/ai-services/openai/concepts/use-your-data)는 모델을 사용자의 데이터와 연결하여 모델의 출력을 향상시키는 방식으로 데이터를 검색하고 활용할 수 있는 기능을 제공합니다. 사용자 입력 및 제공된 대화 기록을 기반으로 지정된 데이터 원본에서 데이터를 검색하고 Azure Cognitive Search와 함께 데이터를 검색합니다. 그런 다음 데이터가 보강되어 모델에 프롬프트로 다시 제출되어 모델이 응답을 생성하는 데 사용할 수 있는 컨텍스트 정보를 제공합니다.

자세한 내용은 [Azure OpenAI 서비스의 데이터, 개인 정보 및 보안](https://learn.microsoft.com/legal/cognitive-services/openai/data-privacy?context=%2Fazure%2Fai-services%2Fopenai%2Fcontext%2Fcontext)을 참조하세요.

## 전제 조건
시작하기 위해 몇 가지 전제 조건을 살펴보겠습니다.

Azure OpenAI 서비스에 제대로 액세스하려면 [Azure 포털](https://portal.azure.com)에서 적절한 리소스를 만들어야 합니다(자세한 방법은 [Microsoft 문서](https://learn.microsoft.com/azure/cognitive-services/openai/how-to/create-resource?pivots=web-portal)에서 확인할 수 있습니다).

Azure OpenAI 모델에 자체 데이터를 사용하려면 다음이 필요합니다:

1. Azure OpenAI 액세스 권한과 채팅 모델이 배포된 리소스(예: GPT-3 또는 GPT-4)
2. Azure 인지 검색 리소스
3. Azure Blob 저장소 리소스
4. 데이터로 사용할 문서([데이터 원본 옵션](https://learn.microsoft.com/azure/ai-services/openai/concepts/use-your-data#data-source-options) 참조)


블로브 스토리지에 문서를 업로드하고 Azure AI Studio를 사용하여 인덱스를 만드는 방법에 대한 전체 안내는 이 [빠른 시작](https://learn.microsoft.com/azure/ai-services/openai/use-your-data-quickstart?pivots=programming-language-studio&tabs=command-line)을 참조하세요.

설정 ## 설정

먼저 필요한 종속성을 설치합니다.

In [ ]:
! pip install "openai>=0.27.6"
! pip install python-dotenv

이 예제에서는 `dotenv`를 사용하여 환경 변수를 로드합니다. Azure OpenAI 및 검색 인덱스와 연결하려면 다음 변수를 `.env` 파일에 `KEY=VALUE` 형식으로 추가해야 합니다:

* `OPENAI_API_BASE` - Azure OpenAI 엔드포인트. 이 파일은 Azure 포털의 Azure OpenAI 리소스에 대한 "키 및 엔드포인트"에서 찾을 수 있습니다.
* `OPENAI_API_KEY` - Azure OpenAI API 키. 이 키는 Azure 포털의 Azure OpenAI 리소스에 대한 "키 및 끝점"에서 찾을 수 있습니다. Azure Active Directory 인증을 사용하는 경우 생략합니다(아래 'Microsoft Active Directory를 사용한 인증' 참조).
* `SEARCH_ENDPOINT` - 코그니티브 검색 엔드포인트. 이 URL은 Azure 포털의 검색 리소스의 '개요'에서 찾을 수 있습니다.
* `SEARCH_KEY` - 코그니티브 검색 API 키. Azure Portal에서 검색 리소스의 '키'에서 찾을 수 있습니다.
* 검색 인덱스 이름` - 자체 데이터로 만든 인덱스의 이름입니다.

In [ ]:
import os
import openai
import dotenv

dotenv.load_dotenv()

In [2]:
openai.api_base = os.environ["OPENAI_API_BASE"]

# Azure OpenAI on your own data is only supported by the 2023-08-01-preview API version
openai.api_version = "2023-08-01-preview"

인증 ### 인증

Azure OpenAI 서비스는 API 키 및 Azure 자격 증명을 포함하는 여러 인증 메커니즘을 지원합니다.

In [3]:
use_azure_active_directory = False  # Set this flag to True if you are using Azure Active Directory


#### API 키를 사용한 인증

OpenAI SDK가 *Azure API 키*를 사용하도록 설정하려면 `api_type`을 `azure`로 설정하고 `api_key`를 엔드포인트와 연결된 키로 설정해야 합니다(이 키는 [Azure 포털](https://portal.azure.com)의 *"리소스 관리"* 아래 "키 및 엔드포인트"*에서 찾을 수 있음).

In [4]:
if not use_azure_active_directory:
    openai.api_type = 'azure'
    openai.api_key = os.environ["OPENAI_API_KEY"]

#### Microsoft Active Directory를 사용한 인증
이제 Microsoft Active Directory 인증을 통해 키를 받는 방법을 알아봅시다. 설정 방법에 대한 자세한 내용은 [문서](https://learn.microsoft.com/azure/ai-services/openai/how-to/managed-identity)를 참조하세요.

In [ ]:
! pip install azure-identity

In [5]:
from azure.identity import DefaultAzureCredential

if use_azure_active_directory:
    default_credential = DefaultAzureCredential()
    token = default_credential.get_token("https://cognitiveservices.azure.com/.default")

    openai.api_type = "azure_ad"
    openai.api_key = token.token

토큰은 일정 기간 동안 유효하며 그 이후에는 만료됩니다. 모든 요청에 유효한 토큰이 전송되도록 하려면 요청.auth에 연결하여 만료되는 토큰을 새로 고칠 수 있습니다:

In [8]:
import typing
import time
import requests

if typing.TYPE_CHECKING:
    from azure.core.credentials import TokenCredential

class TokenRefresh(requests.auth.AuthBase):

    def __init__(self, credential: "TokenCredential", scopes: typing.List[str]) -> None:
        self.credential = credential
        self.scopes = scopes
        self.cached_token: typing.Optional[str] = None

    def __call__(self, req):
        if not self.cached_token or self.cached_token.expires_on - time.time() < 300:
            self.cached_token = self.credential.get_token(*self.scopes)
        req.headers["Authorization"] = f"Bearer {self.cached_token.token}"
        return req


자체 데이터를 사용한 ## 채팅 완료 모델

컨텍스트 설정하기 ###

이 예제에서는 모델이 Azure AI 서비스 설명서 데이터를 기반으로 응답하도록 합니다. 이전에 공유한 [빠른 시작](https://learn.microsoft.com/azure/ai-services/openai/use-your-data-quickstart?tabs=command-line&pivots=programming-language-studio)에 이어 [Azure AI 서비스 및 기계 학습](https://learn.microsoft.com/azure/ai-services/cognitive-services-and-machine-learning) 설명서 페이지에 대한 [마크다운](https://github.com/MicrosoftDocs/azure-docs/blob/main/articles/ai-services/cognitive-services-and-machine-learning.md) 파일을 검색 인덱스에 추가했습니다. 이제 Azure AI 서비스 및 머신 러닝에 대한 질문에 답변할 수 있는 모델이 준비되었습니다.

### 코드

Python SDK로 자체 데이터를 사용하여 Azure OpenAI 모델과 채팅하려면 먼저 자체 데이터로 작동하도록 설계된 채팅 완료 확장 엔드포인트를 대상으로 하는 코드를 설정해야 합니다. 이를 위해 주어진 배포 ID에 대한 확장 엔드포인트를 대상으로 하는 라이브러리에 대한 사용자 지정 어댑터를 설정하기 위해 호출할 수 있는 편의 함수를 만들었습니다.

In [5]:
import requests

def setup_byod(deployment_id: str) -> None:
    """Sets up the OpenAI Python SDK to use your own data for the chat endpoint.
    
    :param deployment_id: The deployment ID for the model to use with your own data.

    To remove this configuration, simply set openai.requestssession to None.
    """

    class BringYourOwnDataAdapter(requests.adapters.HTTPAdapter):

        def send(self, request, **kwargs):
            request.url = f"{openai.api_base}/openai/deployments/{deployment_id}/extensions/chat/completions?api-version={openai.api_version}"
            return super().send(request, **kwargs)

    session = requests.Session()

    # Mount a custom adapter which will use the extensions endpoint for any call using the given `deployment_id`
    session.mount(
        prefix=f"{openai.api_base}/openai/deployments/{deployment_id}",
        adapter=BringYourOwnDataAdapter()
    )

    if use_azure_active_directory:
        session.auth = TokenRefresh(default_credential, ["https://cognitiveservices.azure.com/.default"])

    openai.requestssession = session


이제 편의 함수를 호출하여 자체 데이터에 사용하려는 모델로 SDK를 구성할 수 있습니다.

In [6]:
setup_byod("gpt-4")

데이터 소스` 키워드 인수에 대한 검색 엔드포인트, 키 및 인덱스 이름을 제공하면 이제 모델에 제기된 모든 질문이 자체 데이터에 근거하게 됩니다. 추가 속성인 `context`가 제공되어 모델이 질문에 답하기 위해 참조한 데이터를 표시합니다.

In [7]:
completion = openai.ChatCompletion.create(
    messages=[{"role": "user", "content": "What are the differences between Azure Machine Learning and Azure AI services?"}],
    deployment_id="gpt-4",
    dataSources=[  # camelCase is intentional, as this is the format the API expects
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": os.environ["SEARCH_ENDPOINT"],
                "key": os.environ["SEARCH_KEY"],
                "indexName": os.environ["SEARCH_INDEX_NAME"],
            }
        }
    ]
)
print(completion)

{
  "id": "65b485bb-b3c9-48da-8b6f-7d3a219f0b40",
  "model": "gpt-4",
  "created": 1693338769,
  "object": "extensions.chat.completion",
  "choices": [
    {
      "index": 0,
      "finish_reason": "stop",
      "message": {
        "role": "assistant",
        "content": "Azure AI services and Azure Machine Learning (AML) both aim to apply artificial intelligence (AI) to enhance business operations, but they target different audiences and offer different capabilities [doc1]. \n\nAzure AI services are designed for developers without machine learning experience and provide pre-trained models to solve general problems such as text analysis, image recognition, and natural language processing [doc5]. These services require general knowledge about your data without needing experience with machine learning or data science and provide REST APIs and language-based SDKs [doc2].\n\nOn the other hand, Azure Machine Learning is tailored for data scientists and involves a longer process of data co

모델에서 응답을 스트리밍하려면 `stream=True` 키워드 인수를 전달하면 됩니다:

In [8]:
response = openai.ChatCompletion.create(
    messages=[{"role": "user", "content": "What are the differences between Azure Machine Learning and Azure AI services?"}],
    deployment_id="gpt-4",
    dataSources=[
        {
            "type": "AzureCognitiveSearch",
            "parameters": {
                "endpoint": os.environ["SEARCH_ENDPOINT"],
                "key": os.environ["SEARCH_KEY"],
                "indexName": os.environ["SEARCH_INDEX_NAME"],
            }
        }
    ],
    stream=True,
)

for chunk in response:
    delta = chunk.choices[0].delta

    if "role" in delta:
        print("\n"+ delta.role + ": ", end="", flush=True)
    if "content" in delta:
        print(delta.content, end="", flush=True)
    if "context" in delta:
        print(f"Context: {delta.context}", end="", flush=True)

Context: {
  "messages": [
    {
      "role": "tool",
      "content": "{\"citations\":[{\"content\":\"<h2 id=\\\"how-are-azure-ai-services-and-azure-machine-learning-aml-similar\\\">How are Azure AI services and Azure Machine Learning (AML) similar?.</h2>\\n<p>Both have the end-goal of applying artificial intelligence (AI) to enhance business operations, though how each provides this in the respective offerings is different..</p>\\n<p>Generally, the audiences are different:</p>\\n<ul>\\n<li>Azure AI services are for developers without machine-learning experience..</li>\\n<li>Azure Machine Learning is tailored for data scientists.\",\"id\":null,\"title\":\"Azure AI services and machine learning\",\"filepath\":\"cognitive-services-and-machine-learning.md\",\"url\":\"https://krpraticstorageacc.blob.core.windows.net/azure-openai/cognitive-services-and-machine-learning.md\",\"metadata\":{\"chunking\":\"orignal document size=1188. Scores=5.689296 and None.Org Highlight count=160.Filtering 